<a href="https://colab.research.google.com/github/Sam9875/whiteboard/blob/main/Another_copy_of_Untitled3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
from pyspark.sql import SparkSession
from itertools import combinations
import os

In [ ]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("AmazonFrequentProductPairs") \
    .getOrCreate()
sc = spark.sparkContext

In [ ]:
# Shell 3: Set file paths and clean previous output
input_path = "/content/ReviewsSample.csv"
output_path = "/content/Lab6_output"

if os.path.exists(output_path):
    os.system(f"rm -rf {output_path}")

In [ ]:
# Shell 4: Read and preprocess the data
data = sc.textFile(input_path)
header = data.first()
reviews = data.filter(lambda line: line != header) \
             .map(lambda line: line.split(",")) \
             .map(lambda fields: (fields[2].strip(), fields[1].strip()))  # (UserId, ProductId)


In [ ]:
user_products = reviews.groupByKey() \
                      .mapValues(lambda products: list(set(products))) \
                      .filter(lambda x: len(x[1]) > 1)  # Only users with ≥2 products

In [ ]:
product_pairs = user_products.flatMap(lambda user_data: list(combinations(user_data[1], 2))) \
                            .map(lambda pair: (tuple(sorted(pair)), 1)) \
                            .reduceByKey(lambda a, b: a + b) \
                            .filter(lambda x: x[1] > 1)  # Only pairs with frequency > 1

In [ ]:
sorted_pairs = product_pairs.map(lambda x: (x[1], x[0])) \
                           .sortByKey(ascending=False) \
                           .map(lambda x: (x[1][0], x[1][1], x[0]))

sorted_pairs.saveAsTextFile(output_path)

In [ ]:
# Shell 8: Show sample results
print("Top 10 frequently reviewed product pairs:")
for pair in sorted_pairs.take(10):
    print(f"{pair[0]} | {pair[1]} | Frequency: {pair[2]}")


Top 10 frequently reviewed product pairs:
B1 | B3 | Frequency: 3
B3 | B5 | Frequency: 3
B1 | B5 | Frequency: 3
